In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
import os
import glob
import random
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score
import logging

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("PyTorch version:", torch.__version__)
print("CUDA is available:", torch.cuda.is_available())


PyTorch version: 2.6.0+cu118
CUDA is available: True


In [2]:
def preprocess_images(df):
    new_df = df.copy()
    deleted_count = 0
    valid_indices = []
    for idx, row in tqdm(new_df.iterrows(), total=len(new_df)):
        path = row["path"]
        try:
            img = Image.open(path).convert("RGB")
            img.verify()
            valid_indices.append(idx)
        except (UnidentifiedImageError, FileNotFoundError, OSError) as e:
            logging.info(f"Image error: {path} ({str(e)})")
            try:
                if os.path.exists(path):
                    os.remove(path)
                    deleted_count += 1
                    logging.info(f"Deleted image: {path}")
            except (PermissionError, OSError) as e:
                logging.error(f"Cannot delete image {path}: {str(e)}")
    new_df = new_df.loc[valid_indices].reset_index(drop=True)
    print(f"Deleted {deleted_count} invalid labeled images. {len(new_df)} images remain.")
    logging.info(f"Deleted {deleted_count} labeled images. {len(new_df)} images remain.")
    return new_df

In [3]:
def preprocess_unlabeled_images(image_paths):
    new_image_paths = []
    deleted_count = 0
    for path in tqdm(image_paths):
        try:
            img = Image.open(path).convert("RGB")
            img.verify()
            new_image_paths.append(path)
        except (UnidentifiedImageError, FileNotFoundError, OSError) as e:
            logging.info(f"Image error: {path} ({str(e)})")
            try:
                if os.path.exists(path):
                    os.remove(path)
                    deleted_count += 1
                    logging.info(f"Deleted image: {path}")
            except (PermissionError, OSError) as e:
                logging.error(f"Cannot delete image {path}: {str(e)}")
    print(f"Deleted {deleted_count} invalid unlabeled images. {len(new_image_paths)} images remain.")
    logging.info(f"Deleted {deleted_count} unlabeled images. {len(new_image_paths)} images remain.")
    return new_image_paths

image_dir = r"F:/unlabeled_images"
image_paths = []
for ext in ["*.jpg", "*.jpeg", "*.png", "*.bmp"]:
    image_paths.extend(glob.glob(os.path.join(image_dir, "**", ext), recursive=True))
print(f"Total unlabeled images before check: {len(image_paths)}")
image_paths = preprocess_unlabeled_images(image_paths)
print(f"Images after check and error removal: {len(image_paths)}")


label_csv_path = r"F:\Soil_Labeled_Data\labels.csv"
fallback_dir = r"F:\Soil_Labeled_Data\augmented_fallback"
os.makedirs(fallback_dir, exist_ok=True)
df = pd.read_csv(label_csv_path)
print(f"Total labeled images before check: {len(df)}")
print("Sample image paths:")
print(df["path"].head())
df = preprocess_images(df)

augment = transforms.ColorJitter(brightness=0.2, contrast=0.2)
replaced_count = 0
print("Checking and replacing invalid images...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    path = row["path"]
    try:
        img = Image.open(path).convert("RGB")
        img.verify()
    except (UnidentifiedImageError, FileNotFoundError, OSError):
        print(f"Image error: {path}")
        folder = os.path.dirname(path)
        all_images = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".png"))]
        good_images = [f for f in all_images if f != os.path.basename(path)]
        if not good_images:
            print(f"No replacement images in: {folder}")
            continue
        candidate = random.choice(good_images)
        candidate_path = os.path.join(folder, candidate)
        try:
            img = Image.open(candidate_path).convert("RGB")
            img_aug = augment(img)
            new_filename = f"aug_{os.path.basename(path)}"
            new_path = os.path.join(fallback_dir, new_filename)
            img_aug.save(new_path)
            df.at[idx, "path"] = new_path
            replaced_count += 1
            print(f"Replaced with: {new_path}")
        except Exception as e:
            print(f"Replacement image error: {candidate_path} ({e})")
print(f"Replaced {replaced_count} invalid images with augmented versions.")

image_size = 224
transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

labeled_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class UnlabeledImageDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert("RGB")
            img1 = self.transform(image)
            img2 = self.transform(image)
            return img1, img2
        except Exception as e:
            print(f"Error reading image {img_path}: {e}")
            logging.error(f"Error reading image {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self.image_paths))

class LabeledImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        try:
            image = Image.open(img_path).convert("RGB")
            img = self.transform(image)
        except Exception as e:
            print(f"Error reading image {img_path}: {e}")
            logging.error(f"Error reading image {img_path}: {e}")
            return self.__getitem__((idx + 1) % len(self.df))
        humidity = torch.tensor([row['SM_0'] / 100, row['SM_20'] / 100], dtype=torch.float32)
        class_label = torch.tensor(row["moisture_class"], dtype=torch.long)
        return img, humidity, class_label




Total unlabeled images before check: 11995


100%|███████████████████████████████████████████████████████████████████████████| 11995/11995 [01:25<00:00, 140.20it/s]


Deleted 0 invalid unlabeled images. 11995 images remain.
Images after check and error removal: 11995
Total labeled images before check: 2057
Sample image paths:
0    F:\Soil_Labeled_Data\M_0_10\L_0_10\alluvial\z5...
1    F:\Soil_Labeled_Data\M_0_10\L_0_10\alluvial\z5...
2    F:\Soil_Labeled_Data\M_0_10\L_0_10\alluvial\z5...
3    F:\Soil_Labeled_Data\M_0_10\L_0_10\alluvial\z5...
4    F:\Soil_Labeled_Data\M_0_10\L_0_10\alluvial\z5...
Name: path, dtype: object


100%|██████████████████████████████████████████████████████████████████████████████| 2057/2057 [01:07<00:00, 30.47it/s]


Deleted 0 invalid labeled images. 2052 images remain.
Checking and replacing invalid images...


100%|██████████████████████████████████████████████████████████████████████████████| 2052/2052 [01:08<00:00, 30.17it/s]

Replaced 0 invalid images with augmented versions.


In [4]:
unlabeled_dataset = UnlabeledImageDataset(image_paths, transform)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=8, shuffle=True, drop_last=True, num_workers=0)
labeled_dataset = LabeledImageDataset(df, labeled_transform)
labeled_dataloader = DataLoader(labeled_dataset, batch_size=8, shuffle=True, drop_last=True, num_workers=0)

In [5]:
class SoilNetDualHead(nn.Module):
    def __init__(self, num_classes=10, simclr_mode=False):
        super().__init__()
        self.simclr_mode = simclr_mode
        self.initial_conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.mnv2_block1 = nn.Sequential(*list(
            timm.create_model("mobilenetv2_100.ra_in1k", pretrained=True).blocks.children())[0:3]
        )
        self.channel_adapter = nn.Conv2d(32, 16, kernel_size=1, bias=False)
        self.mobilevit_full = timm.create_model("mobilevitv2_050", pretrained=True)
        self.mobilevit_encoder = self.mobilevit_full.stages
        self.mvit_to_mnv2 = nn.Conv2d(256, 32, kernel_size=1, bias=False)
        self.mnv2_block2 = nn.Sequential(*list(
            timm.create_model("mobilenetv2_100.ra_in1k", pretrained=True).blocks.children())[3:7]
        )
        self.final_conv = nn.Conv2d(320, 1280, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.light_dense = nn.Sequential(nn.Linear(1, 32), nn.ReLU(inplace=True))
        self.reg_head = nn.Sequential(
            nn.Linear(1280 + 32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2)
        )
        self.cls_head = nn.Sequential(
            nn.Linear(1280 + 32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )

    def forward(self, x_img, x_light=None):
        x = self.initial_conv(x_img)
        x = self.mnv2_block1(x)
        x = self.channel_adapter(x)
        x = self.mobilevit_encoder(x)
        x = self.mvit_to_mnv2(x)
        x = self.mnv2_block2(x)
        x = self.final_conv(x)
        x = self.pool(x)
        x_img_feat = torch.flatten(x, 1)
        if self.simclr_mode:
            return x_img_feat
        x_light_feat = self.light_dense(x_light)
        x_concat = torch.cat([x_img_feat, x_light_feat], dim=1)
        reg_out = self.reg_head(x_concat)
        cls_out = self.cls_head(x_concat)
        return reg_out, cls_out


In [6]:
class Projector(nn.Module):
    def __init__(self, input_dim=1280, proj_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.ReLU(inplace=True),
            nn.Linear(input_dim, proj_dim)
        )
    def forward(self, x):
        return self.net(x)

class OnlineLinearRegression(nn.Module):
    def __init__(self, input_dim=1280, output_dim=2):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        return self.linear(x)

class OnlineClassifier(nn.Module):
    def __init__(self, input_dim=1280, num_classes=10):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    def forward(self, x):
        return self.linear(x)

def vicreg_loss(z1, z2, lambda_=25.0, mu=25.0, nu=1.0, epsilon=1e-4):
    invariance_loss = F.mse_loss(z1, z2)
    def variance_term(z):
        z_std = torch.sqrt(z.var(dim=0) + epsilon)
        return torch.mean(F.relu(1 - z_std))
    var_loss = variance_term(z1) + variance_term(z2)
    def covariance_term(z):
        z = z - z.mean(dim=0)
        cov = (z.T @ z) / (z.shape[0] - 1)
        off_diag = cov - torch.diag(cov.diag())
        return off_diag.pow(2).sum() / z.shape[1]
    cov_loss = covariance_term(z1) + covariance_term(z2)
    return lambda_ * invariance_loss + mu * var_loss + nu * cov_loss

model = SoilNetDualHead(num_classes=10, simclr_mode=True).to(device)
checkpoint_path = r"C:\Users\PC\soilNet\Model\Soilnet_pretrained_ImageNet.pth"
try:
    state_dict = torch.load(checkpoint_path, map_location=device)
    filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith(('light_dense', 'reg_head', 'cls_head'))}
    model.load_state_dict(filtered_state_dict, strict=False)
    print("Loaded backbone weights from soilnet_pretrained_imageNet.pth (ignoring light_dense, reg_head, cls_head)")
except FileNotFoundError:
    print("Checkpoint file not found, starting with random weights")


projector = Projector(input_dim=1280, proj_dim=128).to(device)
linear_reg = OnlineLinearRegression(input_dim=1280, output_dim=2).to(device)
classifier = OnlineClassifier(input_dim=1280, num_classes=10).to(device)


optimizer_vicreg = torch.optim.Adam(list(model.parameters()) + list(projector.parameters()), lr=1e-4)
optimizer_linear = torch.optim.Adam(linear_reg.parameters(), lr=1e-3)
optimizer_classifier = torch.optim.Adam(classifier.parameters(), lr=1e-3)

checkpoint_dir = r"C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet"

os.makedirs(checkpoint_dir, exist_ok=True)

Loaded backbone weights from soilnet_pretrained_imageNet.pth (ignoring light_dense, reg_head, cls_head)


In [7]:
def train_vicreg_with_mu(mu=25.0, num_epochs=50):
    vicreg_losses, mse_losses, rmse_losses, mae_losses, accuracy_scores, f1_scores = [], [], [], [], [], []
    labeled_iterator = iter(labeled_dataloader)
    for epoch in range(1, num_epochs + 1):
        model.train()
        projector.train()
        linear_reg.train()
        classifier.train()
        running_vicreg_loss = running_mse = running_mae = running_accuracy = running_f1 = 0.0
        num_batches = 0
        # Gán vòng lặp tqdm vào biến loop
        loop = tqdm(unlabeled_dataloader, leave=False)
        for img1, img2 in loop:
            img1, img2 = img1.to(device), img2.to(device)
            feat1 = model(img1, x_light=None)
            feat2 = model(img2, x_light=None)
            z1 = projector(feat1)
            z2 = projector(feat2)
            vicreg_loss_val = vicreg_loss(z1, z2, mu=mu)
            optimizer_vicreg.zero_grad()
            vicreg_loss_val.backward()
            optimizer_vicreg.step()
            try:
                labeled_img, humidity, class_label = next(labeled_iterator)
            except StopIteration:
                labeled_iterator = iter(labeled_dataloader)
                labeled_img, humidity, class_label = next(labeled_iterator)
            labeled_img, humidity, class_label = labeled_img.to(device), humidity.to(device), class_label.to(device)
            with torch.no_grad():
                feat = model(labeled_img, x_light=None)
            pred_humidity = linear_reg(feat)
            mse_loss = F.mse_loss(pred_humidity, humidity)
            optimizer_linear.zero_grad()
            mse_loss.backward()
            optimizer_linear.step()
            pred_logits = classifier(feat)
            cls_loss = F.cross_entropy(pred_logits, class_label)
            optimizer_classifier.zero_grad()
            cls_loss.backward()
            optimizer_classifier.step()
            pred_humidity_np = pred_humidity.detach().cpu().numpy() * 100
            humidity_np = humidity.detach().cpu().numpy() * 100
            mse = mean_squared_error(humidity_np, pred_humidity_np)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(humidity_np, pred_humidity_np)
            pred_classes = torch.argmax(pred_logits, dim=1).detach().cpu().numpy()
            true_classes = class_label.detach().cpu().numpy()
            accuracy = accuracy_score(true_classes, pred_classes)
            f1 = f1_score(true_classes, pred_classes, average='weighted')
            running_vicreg_loss += vicreg_loss_val.item()
            running_mse += mse
            running_mae += mae
            running_accuracy += accuracy
            running_f1 += f1
            num_batches += 1
            # Cập nhật tiến trình với loop.set_postfix
            loop.set_postfix(vicreg_loss=vicreg_loss_val.item(), mse=mse, rmse=rmse, mae=mae, accuracy=accuracy, f1=f1)
        avg_vicreg_loss = running_vicreg_loss / num_batches
        avg_mse = running_mse / num_batches
        avg_rmse = np.sqrt(avg_mse)
        avg_mae = running_mae / num_batches
        avg_accuracy = running_accuracy / num_batches
        avg_f1 = running_f1 / num_batches
        vicreg_losses.append(avg_vicreg_loss)
        mse_losses.append(avg_mse)
        rmse_losses.append(avg_rmse)
        mae_losses.append(avg_mae)
        accuracy_scores.append(avg_accuracy)
        f1_scores.append(avg_f1)
        print(f"Epoch {epoch:3d}/{num_epochs} (mu={mu}) - VICReg Loss: {avg_vicreg_loss:.4f}, MSE: {avg_mse:.4f}, RMSE: {avg_rmse:.4f}, MAE: {avg_mae:.4f}, Accuracy: {avg_accuracy:.8f}, F1-Score: {avg_f1:.8f}")
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'projector_state_dict': projector.state_dict(),
            'linear_reg_state_dict': linear_reg.state_dict(),
            'classifier_state_dict': classifier.state_dict(),
            'vicreg_loss': avg_vicreg_loss,
            'mse_loss': avg_mse,
            'rmse_loss': avg_rmse,
            'mae_loss': avg_mae,
            'accuracy': avg_accuracy,
            'f1_score': avg_f1
        }
        checkpoint_path = os.path.join(checkpoint_dir, f'vicreg_mu_{mu}_epoch_{epoch}.pth')
        torch.save(checkpoint, checkpoint_path)
        print(f"Saved checkpoint: {checkpoint_path}")
        logging.info(f"Saved checkpoint: {checkpoint_path}")
    final_model_path = os.path.join(checkpoint_dir, f'vicreg_model_final_mu_{mu}.pth')
    final_projector_path = os.path.join(checkpoint_dir, f'vicreg_projector_final_mu_{mu}.pth')
    final_linear_reg_path = os.path.join(checkpoint_dir, f'vicreg_linear_reg_final_mu_{mu}.pth')
    final_classifier_path = os.path.join(checkpoint_dir, f'vicreg_classifier_final_mu_{mu}.pth')
    torch.save(model.state_dict(), final_model_path)
    torch.save(projector.state_dict(), final_projector_path)
    torch.save(linear_reg.state_dict(), final_linear_reg_path)
    torch.save(classifier.state_dict(), final_classifier_path)
    print(f"Saved final models (mu={mu}):")
    print(f"  - Model: {final_model_path}")
    print(f"  - Projector: {final_projector_path}")
    print(f"  - Linear Regression: {final_linear_reg_path}")
    print(f"  - Classifier: {final_classifier_path}")
    logging.info(f"Saved final models (mu={mu}): {final_model_path}, {final_projector_path}, {final_linear_reg_path}, {final_classifier_path}")

In [8]:
for mu_val in [23.0]:
    train_vicreg_with_mu(mu=mu_val, num_epochs=60)
#for mu_val in [20.0, 23.0, 25.0, 27.0, 30.0, 33.0]:


Epoch   1/60 (mu=23.0) - VICReg Loss: 28.5835, MSE: 2871.5276, RMSE: 53.5866, MAE: 39.7510, Accuracy: 0.21205804, F1-Score: 0.22086073
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_1.pth


Epoch   2/60 (mu=23.0) - VICReg Loss: 25.6181, MSE: 1247.9046, RMSE: 35.3257, MAE: 28.0879, Accuracy: 0.21956304, F1-Score: 0.23091625
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_2.pth


Epoch   3/60 (mu=23.0) - VICReg Loss: 24.7948, MSE: 1150.3058, RMSE: 33.9162, MAE: 27.0348, Accuracy: 0.21756171, F1-Score: 0.22692986
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_3.pth


Epoch   4/60 (mu=23.0) - VICReg Loss: 24.4781, MSE: 1187.3090, RMSE: 34.4574, MAE: 27.5708, Accuracy: 0.21506004, F1-Score: 0.22299668
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_4.pth


Epoch   5/60 (mu=23.0) - VICReg Loss: 24.1707, MSE: 1250.8195, RMSE: 35.3669, MAE: 28.2571, Accuracy: 0.21264176, F1-Score: 0.22138847
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_5.pth


...

Epoch  56/60 (mu=23.0) - VICReg Loss: 22.8917, MSE: 1305.2872, RMSE: 36.1288, MAE: 29.1823, Accuracy: 0.17436624, F1-Score: 0.17620557
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_56.pth


Epoch  57/60 (mu=23.0) - VICReg Loss: 22.8698, MSE: 1303.1526, RMSE: 36.0992, MAE: 29.1943, Accuracy: 0.17611741, F1-Score: 0.17742344
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_57.pth


Epoch  58/60 (mu=23.0) - VICReg Loss: 22.8627, MSE: 1326.0662, RMSE: 36.4152, MAE: 29.4694, Accuracy: 0.16494330, F1-Score: 0.16796495
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_58.pth


Epoch  59/60 (mu=23.0) - VICReg Loss: 22.8595, MSE: 1312.1270, RMSE: 36.2233, MAE: 29.1887, Accuracy: 0.16802869, F1-Score: 0.17104875
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_59.pth


Epoch  60/60 (mu=23.0) - VICReg Loss: 22.8772, MSE: 1280.2643, RMSE: 35.7808, MAE: 28.8433, Accuracy: 0.17828552, F1-Score: 0.17917699
Saved checkpoint: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_mu_23.0_epoch_60.pth
Saved final models (mu=23.0):
  - Model: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_model_final_mu_23.0.pth
  - Projector: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_projector_final_mu_23.0.pth
  - Linear Regression: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_linear_reg_final_mu_23.0.pth
  - Classifier: C:\Users\PC\soilNet\checkpoints_VicReg_tinyImagenet\vicreg_classifier_final_mu_23.0.pth
